<div align="center">

# **Corto #8**

### _análisis de los programas_

Link al Repo Para Ver Todos los Archivos, Incluyendo los Originales .c: https://github.com/sofigo1010/trex-python

</div>

---

## 1. Cambios hechos al código original

> **Objetivo:** medir **Speedup** y **Eficiencia** sin alterar la lógica del algoritmo, únicamente aumentando la carga y explicitando la medición de tiempo para ambos ejecutables.

### 1.1 Parámetros de entrada (carga de trabajo)
- **Aumentamos el tamaño del input** para provocar trabajo suficiente y observar diferencias claras entre la versión secuencial y la paralela:
  - `N_FRAMES = 1000`
  - `N_OBS    = 200000`

### 1.2 Medición de tiempo (sin cambiar la lógica)
- **Paralelo (`paralelizado.c`)**: se toma el tiempo con `now_ms()` **antes** y **después** de `simular_paralelo(...)`, imprimiendo `Tiempo total simulacion: ... ms`.
- **Secuencial (`secuencial.c`)**: se toma el tiempo con `now_ms()` **antes** y **después** del **bucle principal** que actualiza, verifica colisiones y renderiza, imprimiendo `Time elapsed: ... ms`.

### 1.3 Qué NO se cambió (para mantener comparabilidad)
- **Lógica del juego/simulación**: mismas funciones `actualizar_obstaculos`, `verificar_colisiones`, `renderizar_escena`.
- **Estructura del paralelismo**: se mantienen los `#pragma omp` tal como estaban en el código del grupo anterior.
- **Salida por consola**: los `printf` por frame siguen activos en ambas versiones para no alterar el comportamiento observado.


# Informe de rendimiento inicial con el aumento de inputs de secuencial.c vs paralelizado.c original

**Archivos de resultados:**  
- `original-code/par.log` (paralelo)  
- `original-code/sec.log` (secuencial)

## Resumen de métricas

| Versión     | Tiempo total (ms) | Colisiones |
|-------------|-------------------:|-----------:|
| Paralelo    |            48.709  |        673 | 
| Secuencial  |           423.622  |        698 | 

**Speedup (aceleración):**  
$$
S \;=\; \frac{T_{\text{sec}}}{T_{\text{par}}}
\;=\;
\frac{423.622}{48.709}
\;\approx\;
8.70\times
$$

**Reducción relativa de tiempo:**  
$$
\Delta\% \;=\; \left(1 - \frac{T_{\text{par}}}{T_{\text{sec}}}\right)\times 100
\;=\;
\left(1 - \frac{48.709}{423.622}\right)\times 100
\;\approx\;
88.51\%
$$

## Análisis

### 1) Rendimiento (tiempo)
- **El paralelo es ~8.70× más rápido** en tiempo de pared: 48.709 ms vs 423.622 ms.  
- La mejora es **sustancial** y típica cuando la parte dominante del trabajo es paralelizable y el overhead de hilos queda amortizado.


### 2) Sobre el número de iteraciones y su efecto
- **Más iteraciones totales** (trabajo) suelen **mejorar el speedup observado** porque:
  - Amortizan mejor el **overhead** de creación/sincronización de hilos.
  - Aumentan la **fracción paralelizable efectiva**, haciendo que el término serial pese menos (intuición de la Ley de Amdahl).



In [ ]:
import os
# Limita hilos internos de NumPy/BLAS: aseguramos usar exactamente 3 threads (los nuestros).
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("BLIS_NUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

from concurrent.futures import ThreadPoolExecutor
from dataclasses import dataclass
from typing import Optional, TextIO, Tuple

import numpy as np
import time

FPS = 60
DT = 1.0 / FPS

@dataclass
class Player:
    x: float = 5.0
    y: float = 0.0

def now_ms() -> float:
    return time.perf_counter() * 1000.0

def render_scene(frame: int, p: Player, x_curr: np.ndarray, out: Optional[TextIO] = None) -> None:
    """Salida estilo C; lee obs0 del buffer de lectura (x_curr)."""
    line = f"frame={frame}  player.y={p.y:.2f}  obs0.x={x_curr[0]: .2f}\n"
    if out:
        out.write(line)
    else:
        # print(line, end="")
        pass


def update_obstacles(x_src: np.ndarray, v: np.ndarray, dt: float, x_dst: np.ndarray) -> None:
    np.add(x_src, v * dt, out=x_dst)

def collisions_count(x_curr: np.ndarray, p: Player) -> int:
    if p.y >= 1.0:
        return 0
    return int(np.count_nonzero(np.abs(x_curr - p.x) <= 1.0))

def simulate_parallel(
    x0: np.ndarray,
    v: np.ndarray,
    n_frames: int = 1000,
    jump_h: float = 1.2,
    p_jump: float = 0.25,
    seed: int = 123,
    log_path: Optional[str] = "par_python.log",   
) -> Tuple[int, float]:
    rng = np.random.default_rng(seed)

    x_curr = x0.copy()
    x_next = np.empty_like(x_curr)

    player = Player()
    total_collisions = 0

    f = open(log_path, "w", encoding="utf-8") if log_path else None

    t0 = now_ms()
    with ThreadPoolExecutor(max_workers=3) as pool:
        for frame in range(n_frames):
            player.y = float(jump_h if rng.random() < p_jump else 0.0)
            p_snapshot = Player(player.x, player.y)
            fut_upd = pool.submit(update_obstacles, x_curr, v, DT, x_next)
            fut_col = pool.submit(collisions_count, x_curr, p_snapshot)
            fut_ren = pool.submit(render_scene, frame, p_snapshot, x_curr, f)

            fut_upd.result()
            total_collisions += fut_col.result()
            fut_ren.result()

            x_curr, x_next = x_next, x_curr

    t1 = now_ms()

    if f:
        f.write(f"\nColisiones totales: {total_collisions}\n")
        f.close()

    print(f"Colisiones totales: {total_collisions}")
    if total_collisions < 5:
        print("RESULTADO: SOBREVIVE")
    elif total_collisions > 5:
        print("RESULTADO: MUERE")
    else:
        print("RESULTADO: LIMITE (5 colisiones)")
    print(f"Tiempo total simulacion: {t1 - t0:.3f} ms")

    return total_collisions, (t1 - t0)

if __name__ == "__main__":
    N_FRAMES = 1000
    N_OBS    = 200_000
    JUMP_H   = 1.2
    P_JUMP   = 0.25

    x0 = (np.arange(N_OBS, dtype=np.float32) * 10.0)
    v  = (-5.0 - np.arange(N_OBS, dtype=np.float32))
    simulate_parallel(x0, v, N_FRAMES, JUMP_H, P_JUMP, seed=42, log_path="par_python.log")

Colisiones totales: 689
RESULTADO: MUERE
Tiempo total simulacion: 117.746 ms


# Comparativa de rendimiento: **paralelo en C** vs **paralelo en Python**

**Datos medidos**

| Implementación | Colisiones | Tiempo total (ms) | Frames/s procesados |
|---|---:|---:|---:|
| C (OpenMP, 3 hilos) | 673 | 48.709 | ≈ 20 530 |
| Python (ThreadPool + NumPy, 3 hilos) | 689 | 117.746 | ≈ 8 493 |

> Nota: la diferencia en colisiones proviene de la aleatoriedad y no afecta la comparación de tiempos.

---

## Métricas

**Speedup de C respecto a Python**  
$$
S_{\text{C}\succ\text{Py}} \;=\; \frac{T_{\text{Py}}}{T_{\text{C}}}
\;=\; \frac{117.746}{48.709}
\;\approx\; 2.42\times
$$

**Reducción relativa de tiempo al usar C**  
$$
\Delta\% \;=\; \Bigl(1 - \frac{T_{\text{C}}}{T_{\text{Py}}}\Bigr)\times 100
\;=\; \Bigl(1 - \frac{48.709}{117.746}\Bigr)\times 100
\;\approx\; 58.63\%
$$

**Throughput (frames procesados por segundo, no confundir con el FPS “jugable”)**

- C:
  $$
  \text{FPS}_{\text{proc,C}} \;=\; \frac{1000}{48.709/1000}
  \;\approx\; 20{,}531\ \text{frames/s}
  $$
- Python:
  $$
  \text{FPS}_{\text{proc,Py}} \;=\; \frac{1000}{117.746/1000}
  \;\approx\; 8{,}493\ \text{frames/s}
  $$

---

## Conclusiones rápidas

- **¿Qué tan más eficiente es en bajo nivel (C)?**  
  En esta carga de trabajo C reduce el tiempo **≈ 58.6 %** respecto a Python.

- **¿Qué tan más rápida es la implementación en C?**  
  C es **≈ 2.42×** más rápida que la versión paralela equivalente en Python.

---

## ¿Por qué C gana aquí?

- **Menor sobrecosto de ejecución.** C compila a código nativo y las secciones/loops de OpenMP tienen **overhead muy bajo** por frame.  
- **Mejor explotación del hardware.** El compilador puede **autovectorizar** y aprovechar cachés y registros (SIMD) en los bucles sobre arreglos contiguos.
- **Menos capas entre el código y la CPU.** En Python hay coste por:
  - lanzar y sincronizar **3 tareas por frame** (3 000 *futures* en 1 000 frames),
  - llamadas de funciones de alto nivel y creación de objetos efímeros,
  - coordinación con el **GIL** (aunque NumPy lo libera en operaciones vectorizadas, la orquestación sigue en Python).

El resultado práctico es que, aun con NumPy y threads, la versión en Python paga más administración por unidad de trabajo que la versión en C.

---

## Limitaciones de trabajar en bajo nivel (C)

- **Complejidad y tiempo de desarrollo.** Más código “boilerplate”, manejo manual de memoria y sincronización (riesgo de *data races* y *undefined behavior* si no se cuida).
- **Mantenibilidad.** El código es menos expresivo y más difícil de modificar/extender que una solución en Python/NumPy.
- **Portabilidad/Toolchain.** Dependencia de flags del compilador, versión de OpenMP, y particularidades de arquitectura.
- **Depuración.** Bugs de concurrencia y de memoria son más sutiles y costosos de rastrear.

**Resumen:** C ofrece **mayor rendimiento bruto** (≈ 2.42× en esta prueba) a costa de **mayor complejidad**. Python sacrifica velocidad por **productividad** y facilidad para iterar, aunque con NumPy + threads puede acercarse razonablemente cuando la carga es grande y vectorizable.